В этом разделе мы вернемся к распознаванию рукописных цифр из датасета MNIST и попробуем существенно
улучшить наше решение из раздела 3.6. Начнем снова с загрузки данных и импорта библиотек:

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data 
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

C:\Users\Art\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Этот код скачает датасет MNIST (если вы его еще не импортировали раньше), а затем преобразует правильные ответы из него в one-hot представление: правильными ответами станут векторы размерности десять, в которых одна единица на месте нужной цифры.  

Зададим заглушки для тренировочных данных:

In [2]:
x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])

В разделе 3.6 мы считали вход просто вектором длины 784. Это, конечно, сильно осложняло задачу нейронной сети: значения разных пикселов получались совершенно независимыми друг от друга, и мы полностью теряли информацию о том, какие из них расположены ближе друг к другу и, соответственно, должны больше влиять друг на друга. На этот раз мы будем применять сверточные сети, для которых пространственная структура изображений важна и в которых она постоянно используется. Поэтому переформатируем входной вектор в виде двумерного массива из 28 х 28 пикселов:

In [3]:
x_image = tf.reshape(x, [-1, 28, 28, 1])

Обратите внимание, что, как мы уже объясняли выше, формально массив теперь четырехмерный: первая размерность - 1 соответствует заранее неизвестному размеру мини-батча, а в четвертой размерности мы указали, что в каждом пикселе стоит только одно число. Для цветной картинки, например, в каждом пикселе могли бы стоять три числа, соответствующие интенсивностям красного, зеленого и синего цветов (RGB).  

Дальше нужно создать сверточный слой. Во-первых, мы должны выбрать размер ядра свертки — для этого примера давайте возьмем ядро размера 5x5. Во-вторых, нам нужно определиться с числом фильтров, которые мы будем обучать; пусть на первом слое их будет 32. И, в-третьих, мы уже разобрались с числом каналов в нашем изображении, то есть с тем, сколько чисел задают каждый пиксел: так как датасет черно-белый, цветовой канал всего один. Итак, можно создавать переменные для весов свертки:

In [4]:
W_conv_1 = tf.Variable(tf.truncated_normal([5, 5, 1, 32], stddev=0.1))

Давайте тщательно разберем эту строчку. Функцию tf.truncated_normal мы уже видели: мы инициализируем веса с помощью обрезанного нормального распределения с заданным стандартным отклонением 0.1 и ожиданием 0. Такое распределение выбрано потому, что мы планируем в качестве функции активации использовать ReLU. Массив из четырех чисел на входе — это форма тензора, который мы инициализируем с помощью нормального распределения. Первые два параметра задают размер ядра, третий отвечает за число входных каналов, а четвертый определяет собственно число выходных каналов. По сути, двигая наше окно-фильтр по исходному изображению, мы на выходе получаем вместо одного значения столбик из 32 значений, что можно представить себе как применение 32 разных фильтров.    

Итак, с весами разобрались, осталось только задать свободные члены (biases); несмотря на сложную структуру тензора весов, для свободных членов достаточно отвести всего 32 переменные: для каждого из 32 фильтров, независимо от того, к какой именно области изображения он приложен, результат свертки сдвигается на одно и то же число.

In [5]:
b_conv_1 = tf.Variable(tf.constant(0.1, shape=[32]))

Обратите внимание на то, как мало у нас здесь переменных: весов на сверточном слое всего 5 * 5 * 1 * 32 = 800, да еще 32 свободных члена. Когда мы в разделе 3.6 задавали полносвязный слой для работы с теми же MNIST-изображениями, весов получалось 784 * 10 = 7840; а если бы мы захотели добавить скрытый слой размером 32, как здесь, то на первом слое весов стало бы 784 * 32 = 25 088, что гораздо
больше. Это яркая иллюстрация того, как сверточные сети используют дополнительную информацию о структуре входов для того, чтобы делать такую «абсолютную регуляризацию», объединяя массу весов. Мы знаем, что изображение имеет двумерную структуру, знаем его геометрию и заранее определяем, что хотели бы обрабатывать каждое окно в изображении одними и теми же фильтрами: нам все равно, в какой части картинки будут расположены штрихи, определяющие цифру 5, нужно просто распознать, что это именно 5, а не 8.  

Теперь у нас определены переменные для всех весов сверточного слоя; что с ними делать дальше, TensorFlow знает сам:

In [6]:
conv_1 = tf.nn.conv2d(x_image, W_conv_1, strides=[1, 1, 1, 1], padding="SAME") + b_conv_1

Сама функция tf.nn.conv2d только применяет сверточные фильтры, она делает линейную часть работы, а функцию активации нам нужно задать самостоятельно потом. В качестве функции активации, как и собирались, возьмем ReLU:

In [7]:
h_conv_1 = tf.nn.relu(conv_1)

Итак, слой фильтров с нелинейностью готов. Чтобы соблюсти стандартную архитектуру сверточной сети, осталось только добавить слой субдискретизации:

In [8]:
h_pool_1 = tf.nn.max_pool(h_conv_1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")

Функция tf.nn.max_pool определяет max-pooling слой, выбирая максимальное значение из каждого окна. Параметр ksize здесь как раз и задает размер этого окна, в котором мы выбираем максимальный элемент. Он имеет ту же структуру, что и strides. Обратите внимание, что здесь уже вполне можно представить себе ситуацию, когда первая компонента будет не равна единице и мы захотим выбирать
«самые подходящие» из нескольких последовательных изображений. Можно даже задать первую размерность —1, тогда слой субдискретизации будет выбирать «самое подходящее» изображение из всего мини-батча.  

Параметры strides и padding обозначают здесь то же самое, что и для сверточного слоя, только в этот раз мы двигаемся по изображению в обе стороны с шагом 2. Понятно, что после этого слоя размер изображения в обоих направлениях уменьшится вдвое, до 14 х 14.  

Раз весов у нас не так уж и много, давайте по той же схеме, что и выше, добавим еще один сверточный слой и слой субдискретизации, в этот раз используя на этом слое 64 фильтра:

In [9]:
W_conv_2 = tf.Variable(tf.truncated_normal([5, 5, 32, 64], stddev=0.1))
b_conv_2 = tf.Variable(tf.constant(0.1, shape=[64]))

conv_2 = tf.nn.conv2d(h_pool_1, W_conv_2, strides=[1, 1, 1, 1], padding="SAME") + b_conv_2

h_conv_2 = tf.nn.relu(conv_2)
h_pool_2 = tf.nn.max_pool(h_conv_2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")

Как правило, в глубоких нейронных сетях за сверточными слоями следуют полносвязные, задача которых состоит в том, чтобы «собрать вместе» все признаки из фильтров и собственно перевести их в самый последний слой, который выдаст ответ. Но для начала нам нужно из двумерного слоя сделать плоский; в TensorFlow это делается функцией reshape:

In [10]:
h_pool_2_flat = tf.reshape(h_pool_2, [-1, 7*7*64])

Число 7 x 7 x 64 возникло из-за того, что мы дважды применили субдискретизацию и при этом в последнем слое использовали 64 фильтра. И теперь осталось только добавить полносвязные слои. В этот раз мы не будем подробно расписывать добавление полносвязных слоев, так как это мы уже не раз делали раньше.
Добавляем первый слой из 1024 нейронов:

In [11]:
W_fc_1 = tf.Variable(tf.truncated_normal([7*7*64, 1024], stddev=0.1))
b_fc_1 = tf.Variable(tf.constant(0.1, shape=[1024]))
h_fc_1 = tf.nn.relu(tf.matmul(h_pool_2_flat, W_fc_1) + b_fc_1)

Регуляризуем его дропаутом:

In [12]:
keep_probability = tf.placeholder(tf.float32)
h_fc_1_drop = tf.nn.dropout(h_fc_1, keep_probability)

Теперь добавляем второй, самый последний слой с десятью выходами:

In [13]:
W_fc_2 = tf.Variable(tf.truncated_normal([1024, 10], stddev=0.1))
b_fc_2 = tf.Variable(tf.constant(0.1, shape=[10]))

logit_conv = tf.matmul(h_fc_1_drop, W_fc_2) + b_fc_2
y_conv = tf.nn.softmax(logit_conv)

Осталось определить ошибку и ввести оптимизатор:

In [14]:
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(logits=logit_conv, labels=y))
train_step = tf.train.AdamOptimizer(0.0001).minimize(cross_entropy)

В этот раз мы используем алгоритм оптимизации Adam, о котором уже говорили в разделе 4.5. Оцениваем точность:

In [15]:
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

И осталось только запустить обучение и дождаться результата:

In [16]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
for i in range(10000):
    batch_xs, batch_ys = mnist.train.next_batch(64)
    sess.run(train_step, feed_dict={x: batch_xs, y: batch_ys, keep_probability: 0.5})

In [ ]:
print("Точность: %s" % sess.run(accuracy, 
               feed_dict={x: mnist.test.images, y: mnist.test.labels, keep_probability: 1.}))